In [ ]:
import ipywidgets as widgets
from ipywidgets import Checkbox, VBox, HBox, Label, Text, AppLayout, Button, Layout
from IPython.display import clear_output, display

import weldx
from weldx.constants import WELDX_QUANTITY as Q_
from weldx.welding.groove.iso_9692_1 import get_groove, _groove_name_to_type
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget

In [ ]:
# Layouts
plot_layout = Layout(width="60%", height="550px")
button_layout = Layout(width="200px", height="50px")
textbox_layout = Layout(width="15%", height="30px")
description_layout = Layout(width="30%", height="30px")

In [ ]:
def hbox_FloatText_creator(text, unit, value=7.5):
    hbox = HBox([
        Label(text + " :", layout=description_layout),
        widgets.BoundedFloatText(value=value, min=0, layout=textbox_layout),
        Text(value=unit, placeholder='unit', layout=textbox_layout),
    ])
    return hbox

In [ ]:
# all attributes used in get_groove

# need to hardcode this since not in weldx
FF_code_numbers = ["1.12", "1.13", "2.12", "3.1.1", "3.1.2", "3.1.3", "4.1.1", "4.1.2", "4.1.3"]

attr_list = []
for groove in _groove_name_to_type:
    for attribute, value in _groove_name_to_type[groove]._mapping.items():
        if value not in attr_list:
            attr_list.append(value)

# create dict with hboxes of all attributes
hbox_dict = dict()
for item in attr_list:
    if item == "code_number":
        dropdown = widgets.Dropdown(
            # import those somewhere?
            options=FF_code_numbers,
            layout=description_layout,
        )
        hbox_dict[item] = HBox([Label("code_number :", layout=description_layout), dropdown])
    elif "angle" in item:
        hbox_dict[item] = hbox_FloatText_creator(item, "deg", 45)
    elif "workpiece_thickness" in item:
        hbox_dict[item] = hbox_FloatText_creator(item, "mm", 15)
    else:
        hbox_dict[item] = hbox_FloatText_creator(item, "mm", 5)

groove_list = ["None"]
groove_list.extend(list(_groove_name_to_type.keys()))

groovetype_dropdown = widgets.Dropdown(
    options=groove_list,
    value='None',
    description='Groove type :',
)

selected_values_vbox = VBox([])


o = widgets.Output(layout=plot_layout)

actual_groove = None

with o:
    fig, ax = plt.subplots(1,1, figsize=(5,4), dpi=80)
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.canvas.resizable = False

changer = None
    
def update_plot(change):
    global changer
    changer = change
    if groovetype_dropdown.value != "None":
        with o:
            get_groove_dict = dict()
            get_groove_dict["groove_type"] = groovetype_dropdown.value
            for child in selected_values_vbox.children:
                if child.children[0].value[:-2] == "code_number":
                    get_groove_dict[child.children[0].value[:-2]] = child.children[1].value
                else:
                    get_groove_dict[child.children[0].value[:-2]] = Q_(child.children[1].value, child.children[2].value)
                
            global actual_groove
            actual_groove = get_groove(**get_groove_dict)
            ax.lines = []
            actual_groove.plot(line_style="-", ax=ax)
            
    else:
        with o:
            ax.lines = []
            ax.set_xlabel(None)
            ax.set_ylabel(None)
            ax.set_title(None)


def remove_not_selected(change):
    if groovetype_dropdown.value == "None":
        selected_values_vbox.children = []
    else:
        # select correct sliders
        selected_values_vbox.children = [
            slider for key, slider in hbox_dict.items() 
            if key in [
                _groove_name_to_type[groovetype_dropdown.value]._mapping[x] 
                for x in _groove_name_to_type[groovetype_dropdown.value]._mapping
            ]
        ] 

button_o = widgets.Output()        

def on_button_clicked(b):
    with button_o:
        clear_output()
        tree = {"groove" : actual_groove}
        buffer = weldx.asdf.utils._write_buffer(tree)
        display(weldx.asdf.utils.notebook_fileprinter(buffer))
        
# observe
groovetype_dropdown.observe(remove_not_selected, names='value')
groovetype_dropdown.observe(update_plot, "value")
for key, box in hbox_dict.items():
    box.children[1].observe(update_plot, "value")
    if key != "code_number":
        box.children[2].observe(update_plot, "value")


# button
save_as_asdf = Button(
    description= "Show as .yml File",
    layout=button_layout
)
save_as_asdf.on_click(on_button_clicked)

# create rest
update_plot(None)
groove_selection = VBox([groovetype_dropdown, selected_values_vbox])
result = VBox([HBox([groove_selection, o]), save_as_asdf, button_o])
 
display(result)

In [ ]:
_groove_name_to_type["FFGroove"]